In [ ]:
import argparse
from Bio import SeqIO
import re
import csv
import duckdb

# internal representation
from protein_metadata import ProteinSentence
from protein_metadata import ProteinWord
from protein_db import ProteinDB

# file handles
protein_fasta_file = "/Users/patrick/dev/ucl/comp0158_mscproject/data/uniref100_10M.fasta";
interpro_file = "/Users/patrick/dev/ucl/comp0158_mscproject/data/protein2ipr.dat";

### File Examples

<p>>Uniref100 Fasta
>\>UniRef100_Q197F3 Uncharacterized protein 007R n=1 Tax=Invertebrate iridescent virus 3 TaxID=345201 RepID=007R_IIV3
MEAKNITIDNTTYNFFKFYNINQPLTNLKYLNSERLCFSNAVMGKIVDDASTITITYHRV
YFGISGPKPRQVADLGEYYDVNELLNYDTYTKTQEFAQKYNSLVKPTIDAKNWSGNELVL

<p> Protein2ipr.dat
>A0A000	IPR004839	Aminotransferase, class I/classII	PF00155	41	381<br>
A0A000	IPR010961	Tetrapyrrole biosynthesis, 5-aminolevulinic acid synthase	TIGR01821	12	391<br>
A0A000	IPR015421	Pyridoxal phosphate-dependent transferase, major domain	G3DSA:3.40.640.10	48	288<br>

<p>> Explanation
A0A0 Means its uniprot but not yet verified

### Useful snippets

In [ ]:
#### grep protein id from interpro

In [ ]:
#file = "/Users/patrick/dev/ucl/comp0158_mscproject/data/protein2ipr.dat";
#file = "/Users/patrick/dev/ucl/comp0158_mscproject/data/protein2ipr_pfam.dat"; # protein2ipr with only pfam entries
#file = "/Users/patrick/dev/ucl/comp0158_mscproject/data/protein2ipr_pfam_20M.dat"; # protein2ipr with only pfam entries 0 20M lines only
file = "/Users/patrick/dev/ucl/comp0158_mscproject/data/A8KBH6_ipr_pfam.dat"; # protein2ipr with only pfam entries - only for one protein

#
# greps for an id in interpro dat file
# For A8KBH6, this will match A0A01A8KBH6 and A8KBH6
# Time to parse the full protein2ipr.dat for A8KBH6 : 22min 56s
#
def grep_interpro(id):
    with open(file, 'r') as input_file:
        for line_number, line in enumerate(input_file):
            #match_string = "ç"            # works
            #match_string = "[A0A0-9]*A8KBH6"   # works
            #match_string = "^[A0A0-9]*"+id     # works
            match_string = "^[A0A0-9]*"+id
            match = re.search(match_string, line)
            if match:
                print('Matched:', id, 'in line:', line.strip())
                
grep_interpro('A8KBH6')
grep_interpro('A0A1A8KBH6')

## Data Preparation


#### Create extract file of PFAM entries from Interpro protein2ipr

In [ ]:
#10M lines 10000000 : 20s
#MAX_LINES = 10000000

limit       = True # if True, onLy parses Max_lines lines 
MAX_COUNT   = 25000000

# ------------------------------------------------------------------------------------------
# 26 June 2024
# parses protein2ipr.dat for entries with'PFNNN' and outpts those lines to a separate file
# protein2ipr.dat       : 98.7GB,   1,355,591,115 entries
# protein2ipr_new.dat   : 20.73GB,  298,766,058 entries
# parsing time          : 23mins
# ----------------------------------------------------------------------------------------

input = "/Users/patrick/dev/ucl/comp0158_mscproject/data/protein2ipr.dat"
output = "/Users/patrick/dev/ucl/comp0158_mscproject/data/protein2ipr_pfam_25M.dat"

#
# parse an Interpro file and grep out those with pfam domains
# Parsing the full protein2ipr.dat file in python took: 23mins 40s
#
def create_pfam_interpro():
    match_count  = 0
    output_file = open(output, "w")

    with open(input, 'r') as input_file:
        for line_number, line in enumerate(input_file):

            # just match for PF
            match = re.search("PF[0-9]+", line) 
            
            if match:
                match_count += 1
                if match_count > MAX_COUNT:
                    print(MAX_COUNT, 'limit reached, breaking.')
                    break

                output_file.write(line)
    output_file.close()

#create_pfam_interpro()

### DATABASE

##### Create db tables

In [ ]:
db = ProteinDB()
db.create_tables()
db.describe_tables()

#### Create indexes after data load

In [ ]:
db.create_indexes()

#### Drop Tables

In [ ]:
con = duckdb.connect(database=ProteinDB.db_string)           
result = con.execute("DROP TABLE PROTEIN_SENTENCE")
con.close()

In [ ]:
con = duckdb.connect(database=ProteinDB.db_string)           
#result = con.execute("DROP TABLE PROTEIN_SENTENCE")

con.execute("\
    CREATE TABLE PROTEIN_SENTENCE (\
        UNIPROT_ID VARCHAR,\
        IPR VARCHAR,\
        DESCRIPTION VARCHAR,\
        TAX_NAME VARCHAR,\
        TAX_ID VARCHAR,\
        DOM_TYPE VARCHAR,\
        START_POS USMALLINT,\
        END_POS USMALLINT\
            )")
con.close()

#### Read PFAM values into DB

In [ ]:
# 10M lines 10000000 took 20s

PROCESS_LIMIT   = 1000000 # number of lines to process, set to -1 to ignore
OUTPUT_LIMIT    = 10000  # determines how often to print a progress message

# DuckDB was loading about 10k rows ever 20s
# 10M rows took 27min and created 83MB on disk - but this was with an index in place, prob quicker
# to apply the index afterwards

#file = "/Users/patrick/dev/ucl/comp0158_mscproject/data/protein2ipr.dat"; # full file
#file = "/Users/patrick/dev/ucl/comp0158_mscproject/data/protein2ipr_new.dat"; # pfam only
file = "/Users/patrick/dev/ucl/comp0158_mscproject/data/protein2ipr_pfam_25M.dat"; # pfam only
#file = "/Users/patrick/dev/ucl/comp0158_mscproject/data/A8KBH6_ipr_pfam.dat"; # as above but only entries for A8KBH6 (pfam only)

# holds a sentence for each sequence
sentences = {}

#
# parse an Interpro file and create ProteinSentences and Words
#
def parse_interpro():
    count = 0
    con = duckdb.connect(database=ProteinDB.db_string)  
    with open(file, 'r') as input_file:
        for line_number, line in enumerate(input_file):
            
            # check if want to output progress
            if ((line_number // OUTPUT_LIMIT > 0) and (line_number % OUTPUT_LIMIT) == 0):
                count += 1
                print(count * OUTPUT_LIMIT, 'lines processed.....')
                 
            # check if we've hit the line limit
            if(PROCESS_LIMIT != -1):
                if line_number == PROCESS_LIMIT:
                    print('Last line:', line)
                if line_number > PROCESS_LIMIT:
                    print('Processing limit reached %s stopping' % (PROCESS_LIMIT))
                    break
        
            # NB: Make sure the file is tab delimited

            # Works for both in: A8KBH6_ipr_pfam.dat
            match = re.search("([A0A0-9]*[a-zA-Z0-9]+)\\tIPR[0-9]+\\t.*\\t(PF[0-9]+)\\t([0-9]+)\\t([0-9]+)", line)
                      
            if match is not None:
                uniprot_id  = match.group(1)
                pfam_word   = match.group(2)
                start       = match.group(3)
                end         = match.group(4)
                item_type   = "PFAM"
                
                #print(uniprot_id, '\t', pfam_word, '\t', start, '\t', end)
                
                '''
                # create a new word item
                word = ProteinWord('pfam', pfam_word, start, end)
                # check if already have a protein with this id
                if (id in sentences.keys()):
                    sentences[id].add_word(word)
                else:
                    sentence = ProteinSentence(id, word)
                    sentences[id] = sentence
                '''
                
                # put into db....
                con.execute("INSERT INTO PROTEIN_WORD (UNIPROT_ID, WORD_TYPE, REF_ID, START_POS, END_POS) VALUES(?,?,?,?,?)", (uniprot_id, item_type, pfam_word, start, end))
    con.close()
                
parse_interpro()


In [ ]:
# test the connection worked
con = duckdb.connect(database=ProteinDB.db_string)           
#id = 'A0A059X392'
id = 'A0A009GV07' # this has multiple pfam entries
result = con.execute("SELECT * FROM PROTEIN_WORD WHERE UNIPROT_ID = ?", [id]).fetchall()
print(result)

#print(sentences['A0A009GV07'])

### UNIREF FASTA

#### Parse UniRef100 FASTA 1 > Check for ProteinSentences

In [ ]:
# 500k is enough for matches to be found
MAX_LINES = 10000000
matched = []
not_matched = []

#
# parse a fasta file to get protein ids
# for uniref, these ids are the characters after UniRef100_
# TODO: Check if ids are proteins or protein clusters
#
def parse_fasta():
    with open(protein_fasta_file, 'r') as input_file:
        for line_number, line in enumerate(input_file):
            
            if line_number > MAX_LINES:  # line_number starts at 0.
                break
            #print('Processing :', line)
            # note that the raw interpro file is tab delimited between fields
            match = re.search("UniRef100_([A-Z0-9]+) ", line)
            if match is not None:
                id = match.group(1)

                if id in sentences.keys():
                    #print(line_number, 'Found sentence for protein :', id, sentences[id].text)
                    if id not in matched:
                        matched.append(id)
                else:
                    not_matched.append(id)
parse_fasta()

#### Parse UniRef100 FASTA 2 > Extract id, length, taxonomy

In [ ]:
'''
# 500k is enough for matches to be found
MAX_LINES = 1000
matched = []
not_matched = []

file = "/Users/patrick/dev/ucl/comp0158_mscproject/data/uniref100_500M.fasta"

#
# parse a fasta file to get protein ids
# for uniref, these ids are the characters after UniRef100_
# TODO: Check if ids are proteins or protein clusters
#
def parse_fasta_2():
    matchline = ""
    id = ""
    match_inprogress = False
    
    with open(file, 'r') as input_file:
        for line_number, line in enumerate(input_file):
            
            if line_number > MAX_LINES:  # line_number starts at 0.
                break

            match = re.search(">UniRef100_([A-Z0-9]+).*TaxID=([0-9]+).*", line)
            
            #match = re.search(">UniRef100_([A-Z0-9]+)", line) # works
            
            if match is not None:
                if(id != ""):
                    print(id, '\t', len(matchline), '\t', tax_id, '\t', matchline)
                matchline = ""
                id = match.group(1)
                tax_id = match.group(2)
                continue
            else:
                matchline += line.strip()
parse_fasta_2()
'''

In [ ]:
print(len(matched),'Matched proteins:\n', matched)
print(len(not_matched), 'Unmatched proteins:\n',not_matched)

#### Parse UniRef100 FASTA 3 > Modified parse_masked_regions from Daniel - outputs masked_regions.dat

In [ ]:
'''
from Bio import SeqIO
import re

# re_string = "\|(.+)\|" # original version
re_string = "UniRef100_([A-Z0-9]+)" # modified for UniRef100

input = "/Users/patrick/dev/ucl/comp0158_mscproject/data/uniref100_10M.fasta"
output = "/Users/patrick/dev/ucl/comp0158_mscproject/data/masked_regions.dat"

def parse_file(path, dom_type):
    output_file = open(output, "w")
    for record in SeqIO.parse(path, "fasta"):
        # record.name = UniRef100_Q6GZX3
        # record.description = UniRef100_Q6GZX3 Putative transc....
        
        # this removes the name from the description removes commas
        raw_desc = record.description.replace(record.name+" ", "")
        raw_desc = raw_desc.replace(",", "")
        
        # extracts the id from the name
        result = re.search(re_string, record.name)
        uniprot_id = result.group(1)
        
        # loops throgh the sequence 3 dots at a time - not sure why
        # is this supposed to return a line for each collection of at least 3 sequence characters?
        # sequence characters?
        #for m in re.finditer(r'\.{3,}', str(record.seq)):          # original version
        for m in re.finditer(r'.{3,}', str(record.seq)):            # modified for UniRef100
            print(uniprot_id+"\tIPRXXXXXX\t"+raw_desc+"\t"+dom_type+"\t" + str(m.start()+1)+"\t"+str(m.end()+1))
            #output_file.write(uniprot_id+"\tIPRXXXXXX\t"+raw_desc+"\t"+dom_type+"\t" + str(m.start()+1)+"\t"+str(m.end()+1) + '\n')
        # return()
    output_file.close()

parse_file(input, "LowComplexity")
#parse_file(file, "CoiledCoil")
'''


#### Read proteins into DB

In [ ]:

def parse_fasta_to_db(dom_type):
    
    path = "/Users/patrick/dev/ucl/comp0158_mscproject/data/uniref100_10M.fasta"
    re_string = "UniRef100_([A-Z0-9]+)" # modified for UniRef100
    
    PROCESS_LIMIT   = 1000 # number of lines to process, set to -1 to ignore
    OUTPUT_LIMIT    = 100  # determines how often to print a progress message
    
    line_number = 0
    count = 0
    con = duckdb.connect(database=ProteinDB.db_string) 
    
    for record in SeqIO.parse(path, "fasta"):
        
        # this removes the name from the description and removes commas
        raw_desc = record.description.replace(record.name+" ", "")
        raw_desc = raw_desc.replace(",", "")
        
        # extracts the id from the name
        result = re.search(re_string, record.name)
        uniprot_id = result.group(1)
        
        # check if want to output progress
        if ((line_number // OUTPUT_LIMIT > 0) and (line_number % OUTPUT_LIMIT) == 0):
            count += 1
            print(count * OUTPUT_LIMIT, 'lines processed.....')
                
        # check if we've hit the line limit
        if(PROCESS_LIMIT != -1):
            if line_number == PROCESS_LIMIT:
                print('Last entry:', uniprot_id)
            if line_number > PROCESS_LIMIT:
                print('Processing limit reached %s stopping' % (PROCESS_LIMIT))
                break
        
        line_number += 1
        
        # loops throgh the sequence 3 dots at a time - not sure why? Is this supposed to 
        # return a line for each collection of at least 3 sequence characters?
        #for m in re.finditer(r'\.{3,}', str(record.seq)):          # original version
        for m in re.finditer(r'.{3,}', str(record.seq)):            # modified for UniRef100
            #print(uniprot_id+"\tIPRXXXXXX\t"+raw_desc+"\t"+dom_type+"\t" + str(m.start()+1)+"\t"+str(m.end()+1))
            ipr="IPRXXXXXX"
            start = str(m.start()+1)
            end = str(m.end()+1)
            con.execute("INSERT INTO PROTEIN_SENTENCE (UNIPROT_ID, IPR, DESCRIPTION, DOM_TYPE, START_POS, END_POS) VALUES(?,?,?,?,?,?,)", (uniprot_id, ipr, raw_desc, dom_type, start, end))
            #output_file.write(uniprot_id+"\tIPRXXXXXX\t"+raw_desc+"\t"+dom_type+"\t" + str(m.start()+1)+"\t"+str(m.end()+1) + '\n')
        # return()
    con.close()

parse_fasta_to_db("LowComplexity")
#parse_file(file, "CoiledCoil")

In [ ]:
# test the connection worked
con = duckdb.connect(database=ProteinDB.db_string)           
#id = 'A0A059X392'
id = 'Q99L13' # this has multiple pfam entries
result = con.execute("SELECT * FROM PROTEIN_SENTENCE WHERE UNIPROT_ID = ?", [id]).fetchall()
print(result)

## DISORDERED REGIONS

In [ ]:
'''
#
# TRIED .gz file directly but too complex
# 

import gzip
import re

input = "/Users/patrick/dev/ucl/comp0158_mscproject/data/disordered/extra.xml.gz"
MAX_LINES = 10

def parse_disordered():
    #pattern = "dbname=\"MOBIDBLT\""
    pattern = ".*"
    
    regex = re.compile(pattern)
    buffer_size = 3
    count = 0

    # Open the gzip file
    with gzip.open(input, 'rt') as file:  # 'rt' mode opens the file in text mode
        buffer = []
        
        for line in file:
            
            protein_match = re.search("<protein id=\"([A-Z0-9]+).*>", line)
            if(protein_match is not None):
                uniprot_id = protein_match.groups(1)
                print('Protein :', uniprot_id, line.strip())
            else:
                db_match = re.search("dbname=\"MOBIDBLT\"([A-Z0-9]+).*>", line)
            
            
            count +=1
            if count >= MAX_LINES:
                break
           
            buffer.append(line)
            # Keep the buffer size within the specified limit
            if len(buffer) > buffer_size:
                buffer.pop(0)
            
            # Join the buffer lines into a single string for regex matching
            combined_lines = ''.join(buffer)
            
            matches = re.search("(.*)", line)
            if matches:
                print('Match :', matches.groups(1), line.strip())

parse_disordered()
'''

#### Read disordered region info into DB

In [52]:
#
# This works directly on the uncompressed .gz file
# No space on laptop for fully extracted extra.xml
#

# Used this command to extract first 10000 lines into a separate file:
#
# zgrep . -m 10000 data/disordered/extra.xml.gz > data/disordered/extra.10000.xml
#

import xml.etree.ElementTree as ElementTree

file = '/Users/patrick/dev/ucl/comp0158_mscproject/data/disordered/extra.10000.xml'

# get an iterable
context = ElementTree.iterparse(file, events=("start", "end"))

# turn it into an iterator
context = iter(context)

# get the root element
event, root = next(context)
con = duckdb.connect(database=ProteinDB.db_string)
 
for event, protein in context:
    if event == "end" and protein.tag == "protein":
        # print(elem.attrib['id'])
        for match in protein:
            if 'MOBIDBLT' in match.attrib['dbname']:
                for coords in match:
                    uniprot_id = protein.attrib['id']
                    word_type="DISORDER"
                    start_pos = coords.attrib['start']
                    end_pos = coords.attrib['end']
                    '''
                    print(protein.attrib['id']+"\tIPRXXXXXX\t" +
                          match.attrib['name']+"\t"+match.attrib['id']+"\t" +
                          coords.attrib['start']+"\t"+coords.attrib['end'])
                    '''
                    con.execute("INSERT INTO PROTEIN_WORD (UNIPROT_ID, WORD_TYPE, START_POS, END_POS) VALUES(?,?,?,?)", (uniprot_id, word_type, start_pos, end_pos))
        # exit()
        root.clear()
con.close()

ParseError: no element found: line 10001, column 0 (<string>)

In [51]:
# test the connection worked
con = duckdb.connect(database=ProteinDB.db_string)           
word_type="DISORDER"
result = con.execute("SELECT * FROM PROTEIN_WORD WHERE WORD_TYPE = ?", [word_type]).fetchall()
print(result)
con.close()

[('A0A001', 'DISORDER', None, None, 312, 340), ('A0A001', 'DISORDER', None, None, 571, 591), ('A0A003', 'DISORDER', None, None, 321, 340), ('A0A007', 'DISORDER', None, None, 368, 395), ('A0A008', 'DISORDER', None, None, 1, 37), ('A0A008', 'DISORDER', None, None, 7, 21), ('A0A009GNL6', 'DISORDER', None, None, 27, 47), ('A0A009GNL6', 'DISORDER', None, None, 33, 47), ('A0A009GPD0', 'DISORDER', None, None, 138, 164), ('A0A009GRX3', 'DISORDER', None, None, 61, 118), ('A0A009GS49', 'DISORDER', None, None, 499, 537), ('A0A009GS49', 'DISORDER', None, None, 511, 531), ('A0A009GT65', 'DISORDER', None, None, 121, 138), ('A0A009GT65', 'DISORDER', None, None, 121, 144), ('A0A009GTE2', 'DISORDER', None, None, 112, 143), ('A0A009GTE2', 'DISORDER', None, None, 115, 129), ('A0A009GU26', 'DISORDER', None, None, 343, 362), ('A0A009GUZ2', 'DISORDER', None, None, 209, 273), ('A0A009GUZ2', 'DISORDER', None, None, 212, 254), ('A0A009GUZ2', 'DISORDER', None, None, 255, 269), ('A0A009GVC4', 'DISORDER', None, N

In [ ]:
# test the connection worked
con = duckdb.connect(database=ProteinDB.db_string)           
word_type="DISORDER"
result = con.execute("SELECT * FROM PROTEIN_WORD WHERE WORD_TYPE = ?", [word_type]).fetchall()
print(result)
con.close()

## Taxonomy

In [ ]:
import csv
import sys
csv.field_size_limit(sys.maxsize)
# parse protein2ipr to output just the pfam domains

#
# get these by extracting taxdump.tar.gz and taxcat.tar.gz
#
names_file = '/Users/patrick/dev/ucl/comp0158_mscproject/data/taxonomy/names.dmp'
cats_file = '/Users/patrick/dev/ucl/comp0158_mscproject/data/taxonomy/categories.dmp'

name_lookup = {}

# print("Reading TAXA names")
with open(names_file) as nf:
    namesreader = csv.reader(nf, delimiter='|', quotechar='\'')
    # each row has 4 cols, but col names have \t - need to remove via rstrip and lstrip
    # cleanrow becomes, fir example : [ 1, root, scientific name, ''] 
    # cleanrow[0] refers to the entry on the first column of the current row
    for row in namesreader:
        clean_row = [x.rstrip().lstrip() for x in row]
        if 'scientific name' in clean_row[3]:
            # print(clean_row)
            clean_row[1] = clean_row[1].replace(',', '')
            
            name_lookup[clean_row[0]] = {}
            name_lookup[clean_row[0]]['name'] = clean_row[1]
            name_lookup[clean_row[0]]['kingdom'] = 'unknown'
            
            
# print("Reading TAXA categories")
with open(cats_file) as cf:
    catreader = csv.reader(cf, delimiter='\t', quotechar='\'')
    for row in catreader:
        # check if item at pos 1 in current row is in namelookup
        if row[1] in name_lookup:
            name_lookup[row[1]]['kingdom'] = row[0]
        if row[2] in name_lookup:
            name_lookup[row[1]]['kingdom'] = row[0]
            
            
'''

# print(name_lookup)
uniprot_lookup = {}
# print("Annotating UNIPROT")
with open('/scratch1/NOT_BACKED_UP/dbuchan/uniprot/idmapping_selected.tab') as mapping:
    mappingreader = csv.reader(mapping, delimiter='\t', quotechar='\'')
    for row in mappingreader:
        if row[12] in name_lookup:
            # print(row)
            uniprot_lookup[row[0]] = name_lookup[row[12]]
            uniprot_lookup[row[0]]['taxaid'] = row[12]
            # break

# print("Annotating UNIPROT PFam")

# print(uniprot_lookup)
with open('/scratch1/NOT_BACKED_UP/dbuchan/interpro/derived/protein2ipr_pfam.dat') as pfam:
# with open('/scratch1/NOT_BACKED_UP/dbuchan/interpro/masked_regions.dat') as pfam:
#with open('/scratch1/NOT_BACKED_UP/dbuchan/interpro/disorder_regions.dat') as pfam:
    pfamreader = csv.reader(pfam, delimiter=',', quotechar='\'')
    for row in pfamreader:
        if row[0] in uniprot_lookup:
            new_line = [row[0], uniprot_lookup[row[0]]['taxaid'],
                        uniprot_lookup[row[0]]['kingdom'],
                        uniprot_lookup[row[0]]['name']] + row[1:]
            print(",".join(new_line))
            sys.stdout.flush()
'''